# Modelo de Risco de Crédito

In [7]:
import pandas as pd

### Extraição de dados usando Kaggle CLI


In [ ]:
%%bash
mkdir -p data && \
cd data && \ 
kaggle competitions download -c home-credit-credit-risk-model-stability && \
unzip home-credit-credit-risk-model-stability.zip && \
rm -rf parquet_files \

### Exploração dos dados

In [ ]:
# Reading Features Definition
df_feat_definition = pd.read_csv('data/feature_definitions.csv')
df_feat_definition
# file = 'train_base.csv'
# file = 'train_static_0_0.csv'
# df = pd.read_csv(f'data/csv_files/train/{file}')
# df.head(10)

                     Variable  \
0              actualdpd_943P   
1     actualdpdtolerance_344P   
2        addres_district_368M   
3            addres_role_871L   
4             addres_zip_823M   
..                        ...   
460  totinstallast1m_4525188A   
461        twobodfilling_608L   
462                  type_25L   
463            typesuite_864L   
464           validfrom_1069D   

                                           Description  
0    Days Past Due (DPD) of previous contract (actu...  
1                        DPD of client with tolerance.  
2                    District of the person's address.  
3                            Role of person's address.  
4                             Zip code of the address.  
..                                                 ...  
460  Total amount of monthly instalments paid in th...  
461                       Type of application process.  
462                          Contact type of a person.  
463  Persons accompanying the cli